In [ ]:
#from env import TradingSPYEnv
#from env2 import TradingSPYEnv
from env6 import TradingSPYEnv
import numpy as np
import os
state_dict_path = os.path.join(os.getcwd(),'test6.pth')
best_state_dict_path = os.path.join(os.getcwd(),'best6.pth')
env = TradingSPYEnv(init_invest=100.0, sma_len=[5,10,15,20,25],train_test_split=0.5)
#num_states = len(env.reset())
num_states = env.reset(50).shape[0]
print("Size of State Space ->  {}".format(num_states))
num_actions = env.action_space.n
print("Size of Action Space ->  {}".format(num_actions))

agent = Agent(gamma=0.99, epsilon=0.5, batch_size=8192, n_actions=num_actions, eps_dec=1e-5, eps_end=0.1, 
              input_dims = [num_states], lr=0.00005, p=0.5,
              weight_decay=1e-5)
#agent.Q_eval.load_state_dict(T.load(state_dict_path))
agent.Q_eval.load_state_dict(T.load(best_state_dict_path))
scores, eps_history = [], []
n_games = 1000
max_score = 0.0
#learn_frequency = 4
#learn_count = 0


for i in range(n_games):
    score = 0.0
    done = False
    observation = env.reset(50)
    while not done:
        action = agent.choose_action(observation)
        observation_, reward, done, info = env.step(action)
        if (observation_ is not None): 
            score += reward
            agent.store_transition(observation, action, reward, observation_, done)
        agent.learn()
#        if learn_count == learn_frequency:
#            agent.learn()
#            learn_count = 0
#        else:
#            learn_count += 1
        observation = observation_
#    agent.learn() # Learn after one episode
    scores.append(score)
    eps_history.append(agent.epsilon)
    
    avg_score = np.mean(scores[-100:])
    
    print('episode ', i, 'score %.2f' % score,
         'average score %.2f' % avg_score,
         'epsilon %.2f ' % agent.epsilon,
         'profit_iteration %.2f ' % info['profit_iteration'],
         'iterations %.2f ' % info['iterations'],
         'long_return %.2f ' % info['long_return']
         )
    
    if i !=0:        
        if (i % 10 == 0):
            T.save(agent.Q_eval.state_dict(), state_dict_path)
            print("saved check point")
        if score > max_score:
            max_score = score
            T.save(agent.Q_eval.state_dict(), best_state_dict_path)
            print("saved best parameters")


In [2]:
import numpy as np
from sac_torch import Agent
from utils import plot_learning_curve
from env6 import TradingSPYEnv
import os

state_dict_path = os.path.join(os.getcwd(),'test6.pth')
best_state_dict_path = os.path.join(os.getcwd(),'best6.pth')
env = TradingSPYEnv(init_invest=100.0, sma_len=[5,10,15,20,25],train_test_split=0.5)
#num_states = len(env.reset())
num_states = env.reset(50).shape[0]
print("Size of State Space ->  {}".format(num_states))
num_actions = env.action_space.n
print("Size of Action Space ->  {}".format(num_actions))

agent = Agent(batch_size=8192, n_actions=num_actions,
              input_dims = [num_states], alpha=0.00005, beta=0.00005, max_action=2,
              reward_scale=1.0, layer1_size=64, layer2_size=64)
#agent.Q_eval.load_state_dict(T.load(state_dict_path))
#agent.Q_eval.load_state_dict(T.load(best_state_dict_path))
scores, eps_history = [], []
n_games = 1000
max_score = 0.0
#learn_frequency = 4
#learn_count = 0


for i in range(n_games):
    score = 0.0
    done = False
    observation = env.reset(50)
    while not done:
        action = agent.choose_action(observation)
        observation_, reward, done, info = env.step(action)
        if (observation_ is not None): 
            score += reward
            agent.store_transition(observation, action, reward, observation_, done)
        agent.learn()
#        if learn_count == learn_frequency:
#            agent.learn()
#            learn_count = 0
#        else:
#            learn_count += 1
        observation = observation_
#    agent.learn() # Learn after one episode
    scores.append(score)
    eps_history.append(agent.epsilon)
    
    avg_score = np.mean(scores[-100:])
    
    print('episode ', i, 'score %.2f' % score,
         'average score %.2f' % avg_score,
         'epsilon %.2f ' % agent.epsilon,
         'profit_iteration %.2f ' % info['profit_iteration'],
         'iterations %.2f ' % info['iterations'],
         'long_return %.2f ' % info['long_return']
         )
    
    if i !=0:        
        if (i % 10 == 0):
            T.save(agent.save_models(), state_dict_path)
            print("saved check point")
        if score > max_score:
            max_score = score
            T.save(agent.save_models(), best_state_dict_path)
            print("saved best parameters")

C:\Users\golde\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Size of State Space ->  200
Size of Action Space ->  3


ValueError: setting an array element with a sequence.

In [4]:
action

array([-0.24178194,  0.3601885 ,  0.02837565], dtype=float32)